In [1]:
import numpy as np
import tensorflow as tf

In [2]:
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

W0628 15:30:38.302362 140443654555456 deprecation.py:323] From <ipython-input-2-1f58289e5867>:11: BasicRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
W0628 15:30:38.303633 140443654555456 deprecation.py:323] From <ipython-input-2-1f58289e5867>:12: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0628 15:30:38.351818 140443654555456 deprecation.py:506] From /home/sean/pyenv/py37/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dt

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

11493376/11490434 [==============================] - 2s 0us/step


In [4]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [5]:
X_test = X_test.reshape((-1, n_steps, n_inputs))

In [6]:
n_epochs = 100
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Last batch accuracy:", acc_batch, "Test accuracy:", acc_test)

0 Last batch accuracy: 0.94 Test accuracy: 0.907
1 Last batch accuracy: 0.93333334 Test accuracy: 0.9495
2 Last batch accuracy: 0.96 Test accuracy: 0.951
3 Last batch accuracy: 0.98 Test accuracy: 0.9649
4 Last batch accuracy: 0.9866667 Test accuracy: 0.9605
5 Last batch accuracy: 0.9866667 Test accuracy: 0.9702
6 Last batch accuracy: 0.98 Test accuracy: 0.971
7 Last batch accuracy: 0.96666664 Test accuracy: 0.9678
8 Last batch accuracy: 0.99333334 Test accuracy: 0.971
9 Last batch accuracy: 0.94 Test accuracy: 0.9538
10 Last batch accuracy: 0.96666664 Test accuracy: 0.9725
11 Last batch accuracy: 0.9866667 Test accuracy: 0.9732
12 Last batch accuracy: 0.97333336 Test accuracy: 0.9748
13 Last batch accuracy: 0.98 Test accuracy: 0.9752
14 Last batch accuracy: 1.0 Test accuracy: 0.9752
15 Last batch accuracy: 0.98 Test accuracy: 0.9746
16 Last batch accuracy: 0.97333336 Test accuracy: 0.9734
17 Last batch accuracy: 0.99333334 Test accuracy: 0.9748
18 Last batch accuracy: 0.9866667 Test a